In [0]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [108]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive


In [0]:
data = pd.read_csv('imdb_master.csv', header = 0, encoding = "latin")

In [0]:
data = data[['type', 'review', 'label']]
data = data[data['label'] != 'unsup']

In [111]:
data.head()

,type,review,label
0,test,Once again Mr. Costner has dragged out a movie...,neg
1,test,This is an example of why the majority of acti...,neg
2,test,"First of all I hate those moronic rappers, who...",neg
3,test,Not even the Beatles could write songs everyon...,neg
4,test,Brass pictures (movies is not a fitting word f...,neg


In [112]:
data.label.unique()

array(['neg', 'pos'], dtype=object)

In [113]:
data.shape

(50000, 3)

In [0]:
pos = []
neg = []
for l in data.label:
    if l == 'neg':
        pos.append(0)
        neg.append(1)
    elif l == 'pos':
        pos.append(1)
        neg.append(0)

In [115]:
print(len(pos), len(neg))
data['Pos']= pos
data['Neg']= neg

50000 50000


In [116]:
data.head()

,type,review,label,Pos,Neg
0,test,Once again Mr. Costner has dragged out a movie...,neg,0,1
1,test,This is an example of why the majority of acti...,neg,0,1
2,test,"First of all I hate those moronic rappers, who...",neg,0,1
3,test,Not even the Beatles could write songs everyon...,neg,0,1
4,test,Brass pictures (movies is not a fitting word f...,neg,0,1


In [0]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['review_clean'] = data['review'].apply(lambda x: remove_punct(x))

In [118]:
import nltk
#from nltk import word_tokenize, WordNetLemmatizer
nltk.download('punkt')
tokens = [word_tokenize(sen) for sen in data.review_clean] 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 

In [120]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stoplist = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [122]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 
print(len(filtered_words))

50000


In [0]:
result = [' '.join(sen) for sen in filtered_words] 

In [124]:
print(len(result))
data['review_final'] = result

50000


In [0]:
data['tokens'] = filtered_words

In [0]:
data = data[['review_final', 'tokens', 'type', 'label', 'Pos', 'Neg']]

In [127]:
data[:4]

,review_final,tokens,type,label,Pos,Neg
0,mr costner dragged movie far longer necessary ...,"[mr, costner, dragged, movie, far, longer, nec...",test,neg,0,1
1,example majority action films generic boring t...,"[example, majority, action, films, generic, bo...",test,neg,0,1
2,first hate moronic rappers couldnt act gun pre...,"[first, hate, moronic, rappers, couldnt, act, ...",test,neg,0,1
3,even beatles could write songs everyone liked ...,"[even, beatles, could, write, songs, everyone,...",test,neg,0,1


In [128]:
data_train = data[data['type'] == 'train']
data_test = data[data['type'] == 'test']
data_train.shape, data_test.shape

((25000, 6), (25000, 6))

In [129]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

3105982 words total, with a vocabulary size of 121722
Max sentence length is 1449


In [130]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

3035376 words total, with a vocabulary size of 119578
Max sentence length is 1191


In [131]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [0]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [0]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [136]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["review_final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["review_final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 121712 unique tokens.


In [0]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [138]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(121713, 300)


In [0]:
test_sequences = tokenizer.texts_to_sequences(data_test["review_final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [0]:
label_names = ['Pos', 'Neg']

In [0]:
y_train = data_train[label_names].values

In [0]:
x_train = train_cnn_data
y_tr = y_train

In [145]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 50, 300)      36513900    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 49, 200)      120200      embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 48, 200)      180200      embedding_2[0][0]                
____________________________________________________________________________________________

In [0]:
num_epochs = 3
batch_size = 34

In [147]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

#implemented following the https://towardsdatascience.com/cnn-sentiment-analysis-1d16b7c5a0e7

Train on 22500 samples, validate on 2500 samples
Epoch 1/3
22500/22500 [==============================] - 132s 6ms/step - loss: 0.5014 - acc: 0.7410 - val_loss: 0.6584 - val_acc: 0.6594
Epoch 2/3
22500/22500 [==============================] - 128s 6ms/step - loss: 0.3566 - acc: 0.8474 - val_loss: 0.3308 - val_acc: 0.8548
Epoch 3/3
22500/22500 [==============================] - 127s 6ms/step - loss: 0.2515 - acc: 0.9007 - val_loss: 0.6040 - val_acc: 0.7340
